# PalData (new)

Data can be retrieved or loaded into the SwarmPAL framework using the `PalData` construct. This construct can hold multiple datasets (in the form of `PalDataItem`'s), connecting in one direction to tools to fetch data, and in the other direction to tools that process and visualise data.

In [ ]:
from datetime import timedelta

from swarmpal.io import PalData, PalDataItem


A `PalData` object is created by passing a series of `PalDataItem` objects. Each `PalDataItem` can be created through different methods...

## PalDataItem from VirES

Pass parameters to the `.from_vires` method to fetch data from VirES (via viresclient)

In [ ]:
item_v = PalDataItem.from_vires(
    server_url="https://vires.services/ows",
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["F", "B_NEC"],
    models=["IGRF"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T00:00:10",
    filters=["(Longitude > 92.8) AND (Latitude < -72.57)"],
    options=dict(asynchronous=False, show_progress=False),
)

The above code will just create the object (with some light validation), without fetching and loading the data. To do that, run `.initialise()`, and then the dataset will be made available in the `.xarray` attribute:

In [ ]:
item_v.initialise()
item_v.xarray

## PalDataItem from HAPI

Pass parameters to the `.from_hapi` method to fetch data from any HAPI server (via hapiclient)

- Note that the available parameters are different from VirES
- Note that the terminology follows the VirES request above,
  rather than the usual HAPI terminology [(see example)](http://hapi-server.org/servers/#server=VirES-for-Swarm&dataset=SW_OPER_MAGA_HR_1B&parameters=B_NEC&start=2013-11-25T11:02:51.642562Z&stop=2013-11-25T11:03:51.642Z&return=script&format=python), i.e.:  
  "collection" = "dataset"  
  "measurements" is a list of values, rather than the comma-separated string "parameters"  
  "start_time", "end_time" = "start", "stop"
- Note that there is not a 1-1 match between data from VirES and HAPI, in terms of metadata (e.g. `B_NEC` dimension labels are currently missing)

In [ ]:
item_h = PalDataItem.from_hapi(
    server_url="https://vires.services/hapi",
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["Latitude", "Longitude", "F", "B_NEC"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T00:00:10",
    options=dict(logging=False),
)
item_h.initialise()
item_h.xarray

## PalDataItem from local file or object

In [ ]:
# Get a copy of the data above to use as a test
ds = item_h.xarray.copy()
ds.to_netcdf("test.nc")
# Create identical Pal objects from that data
item_f = PalDataItem.from_file("test.nc")
item_m = PalDataItem.from_manual(ds)

## Assembling PalData objects

Any number of items can be attached to `PalData` (and the `.initialise()` is passed through to each item).

Items can either be referenced by a simple index:

In [ ]:
my_pal = PalData(item_v, item_h, item_f, item_m)
my_pal.initialise()
my_pal[0], my_pal[1], my_pal[2], my_pal[3]

... or by passing keywords:

In [ ]:
my_pal = PalData(one=item_v, two=item_h, three=item_f, four=item_m)
my_pal.initialise()
my_pal["one"], my_pal["two"], my_pal["three"], my_pal["four"]

Putting the above steps together, we can create a `PalData` directly with:

In [ ]:
my_pal = PalData(
    one=PalDataItem.from_file("test.nc"),
    two=PalDataItem.from_file("test.nc"),
)

... or from a vires/hapi requests directly:

In [ ]:
params = dict(
    server_url="https://vires.services/ows",
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["F", "B_NEC"],
    models=["IGRF"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T00:00:10",
    pad_times=[timedelta(seconds=1), timedelta(seconds=1)],
    options=dict(asynchronous=False, show_progress=False),
)
my_pal = PalData(PalDataItem.from_vires(**params))
my_pal.initialise()
my_pal[0].xarray

### Time padding

Note above that we also supplied the `pad_times` option (a tuple of `datetime.timedelta` instances) that expands the time range being fetched by the specified amounts at the start and end of the window. The original start/end times supplied are available in the `analysis_window` property:

In [ ]:
my_pal[0].analysis_window

In [ ]:
# A final cleanup to remove the file we created earlier
from os import remove

remove("test.nc")